In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rnd
from xgboost import XGBRegressor
from scipy.stats.stats import pearsonr   
from sklearn.metrics import mean_absolute_error

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVC, LinearSVC
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from scipy.stats import skew  # for some statistics
from lightgbm import LGBMRegressor
from scipy.special import boxcox1p
from sklearn.pipeline import make_pipeline
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

X = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
X_test_full = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')


In [3]:
from scipy.stats import norm
from scipy.stats import multivariate_normal as mn

def z_score(df, threshold=1):
    mean, std = np.mean(df), np.std(df)
    z_score = np.abs((df - mean) / std)
    good = z_score < threshold
    print(f"Le Z-score de {threshold} correspond à une probabilité de {100 * 2 * norm.sf(3):0.2f}%")
    print(f"Rejection {(~good).sum()} points")
    return good

In [4]:
def featureEngineering(data, test_data):
        
    # Removing outliers
    filteredData = z_score(data['GrLivArea'], 3)
    data = data[filteredData]
    
    data.drop(['PoolQC'], axis=1, inplace=True)
    data.drop(['Utilities', 'Street'], axis=1, inplace=True)
    
    y = data.SalePrice
    data.drop(['SalePrice'], axis=1, inplace=True)
    
    XConc = pd.concat([data, test_data]).reset_index(drop=True)

    objects = []
    for i in XConc.columns:
        if XConc[i].dtype == object:
            objects.append(i)
    XConc.update(XConc[objects].fillna('None'))

    XConc['LotFrontage'] = XConc.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    XConc['GarageYrBlt'] = XConc.groupby('YearBuilt')['GarageYrBlt'].transform(lambda x: x.fillna(x.median())) # -216 sur la MAE

    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerics = []
    for i in XConc.columns:
        if XConc[i].dtype in numeric_dtypes:
            numerics.append(i)
    data.update(XConc[numerics].fillna(0))
    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerics2 = []
    
    for i in XConc.columns:
        if XConc[i].dtype in numeric_dtypes:
            numerics2.append(i)
    skew_features = XConc[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

    high_skew = skew_features[skew_features > 0.5]
    skew_index = high_skew.index

    for i in skew_index:
        XConc[i] = boxcox1p(XConc[i], boxcox_normmax(XConc[i] + 1))
        
    for col in numerics:
        if np.any(np.isnan(XConc[col])):
            XConc[col] = XConc[col].transform(lambda x: x.fillna(x.median()))

    XConc['YrBltAndRemod']=XConc['YearBuilt']+XConc['YearRemodAdd']
    XConc['FullSF']=XConc['TotalBsmtSF'] + XConc['1stFlrSF'] + XConc['2ndFlrSF']


    XConc['BsmtAndHouseSF'] = (XConc['BsmtFinSF1'] + XConc['BsmtFinSF2'] +  XConc['1stFlrSF'] + XConc['2ndFlrSF'])

    XConc['BathroomsCount'] = (XConc['FullBath'] + (0.5 * XConc['HalfBath']) + XConc['BsmtFullBath'] + (0.5 * XConc['BsmtHalfBath']))

    XConc['PorchSF'] = (XConc['OpenPorchSF'] + XConc['3SsnPorch'] + XConc['EnclosedPorch'] + XConc['ScreenPorch'] +
                          XConc['WoodDeckSF'])

    XConc['haspool'] = XConc['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    XConc['has2ndfloor'] = XConc['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
    XConc['hasgarage'] = XConc['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    XConc['hasbsmt'] = XConc['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
    XConc['hasfireplace'] = XConc['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

    # TODO : trouver des groupby?

    
    return XConc, y

In [5]:
# XGBoost
my_model_2 = XGBRegressor(eta='0.001', n_estimators=1000, learning_rate=0.03, random_state=0, min_child_weight=7, max_depth=5, n_jobs=-1, subsample=0.55,
                              colsample_bytree=0.2, reg_lambda=3.5, alpha=110000)

resXConc, y = featureEngineering(X, X_test_full)
final_features = pd.get_dummies(resXConc).reset_index(drop=True)

#print(np.any(np.isnan(final_features)))

X = final_features.iloc[:len(y), :]
X_sub = final_features.iloc[len(X):, :]

#my_model_2.fit(X, y)


Le Z-score de 3 correspond à une probabilité de 0.27%
Rejection 16 points


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-

In [6]:
pred1 = np.zeros(X.shape[0])
pred2 = np.zeros(X.shape[0])
pred3 = np.zeros(X.shape[0])
pred4 = np.zeros(X.shape[0])

test1 = np.zeros(X_sub.shape[0])
test2 = np.zeros(X_sub.shape[0])
test3 = np.zeros(X_sub.shape[0])
test4 = np.zeros(X_sub.shape[0])

params_lgb = {'reg_alpha': 4.973064761998367, 'reg_lambda': 0.06365096912006087,'colsample_bytree': 0.24,
              'subsample': 0.8, 'learning_rate': 0.015, 'max_depth': 100, 'num_leaves': 43,'min_child_samples': 141,
              'cat_smooth': 18,'metric': 'rmse', 'random_state': 48,'n_estimators': 40000}

kf = KFold(n_splits=5,random_state=48,shuffle=True)
n=0

for trn_idx, test_idx in kf.split(X, y):
    print(f"fold: {n+1}")
    X_tr,X_val = X.iloc[trn_idx],X.iloc[test_idx]
    y_tr,y_val= y.iloc[trn_idx] , y.iloc[test_idx]
    
    
    model1 = lgb.LGBMRegressor(bagging_fraction=1e-10, feature_fraction=0.2)
    model1.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=200)
    pred1[test_idx] = model1.predict(X_val)
    test1 += model1.predict(X_sub)/kf.n_splits
    rmse1 = mean_squared_error(y_val, model1.predict(X_val), squared=False)
    print(": model1 rmse = {}".format(rmse1))
    
    model2 = GradientBoostingRegressor(learning_rate=0.03, n_estimators=800)
    model2.fit(X_tr,y_tr)
    pred2[test_idx] = model1.predict(X_val)
    test2 += model2.predict(X_sub)/kf.n_splits
    rmse2 = mean_squared_error(y_val, model2.predict(X_val), squared=False)
    print(": model2 rmse = {}".format(rmse2))
    
    model3 = BayesianRidge()
    model3.fit(X_tr,y_tr)
    pred3[test_idx] = model1.predict(X_val)
    test3 += model3.predict(X_sub)/kf.n_splits
    rmse3 = mean_squared_error(y_val, model3.predict(X_val), squared=False)
    print(": model3 rmse = {}".format(rmse3))
    
    model4 = XGBRegressor(n_estimators=800, learning_rate=0.03, random_state=0, min_child_weight=7, max_depth=5, n_jobs=-1, subsample=0.5)
    model4.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],verbose=False)
    pred4[test_idx] = model4.predict(X_val)
    test4 += model4.predict(X_sub)/kf.n_splits
    rmse4 = mean_squared_error(y_val, model4.predict(X_val), squared=False)
    print(": model4 rmse = {}".format(rmse4))
    print(": average all models rmse = {}".format((rmse1+rmse2+rmse4)/3))

    n+=1
    
stacked_predictions = np.column_stack((pred1, pred2, pred3, pred4))
stacked_test_predictions = np.column_stack((test1, test2, test3, test4))

l1_train = pd.DataFrame(data={
    "lgbm": pred1.tolist(),
    "GBR": pred2.tolist(),
    "LinearRegression": pred3.tolist(),
    "xgb": pred4.tolist(),
    "target": y
    })
l1_test = pd.DataFrame(data={
    "lgbm": test1.tolist(),
    "GBR": test2.tolist(),
    "LinearRegression": test3.tolist(),
    "xgb": test4.tolist()
    })

kf = KFold(n_splits=5,random_state=48,shuffle=True)
final_prediction = np.zeros(X_sub.shape[0])
rmse=[]
n=0
for trn_idx, test_idx in kf.split(stacked_predictions, y):
    X_tr,X_val=stacked_predictions[trn_idx],stacked_predictions[test_idx]
    y_tr,y_val=y.iloc[trn_idx],y.iloc[test_idx]
    
    meta_model = Lasso()
    
    meta_model.fit(X_tr,y_tr)
    
    final_prediction +=meta_model.predict(stacked_test_predictions)/kf.n_splits
    rmse.append(mean_squared_error(y_val, meta_model.predict(X_val), squared=False))
    print(f"fold: {n+1}, rmse: {rmse[n]}")
    n+=1










fold: 1
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=1e-10, subsample=1.0 will be ignored. Current value: bagging_fraction=1e-10
[1]	valid_0's l2: 3.75473e+09
[2]	valid_0's l2: 3.27266e+09
[3]	valid_0's l2: 2.84298e+09
[4]	valid_0's l2: 2.44014e+09
[5]	valid_0's l2: 2.13086e+09
[6]	valid_0's l2: 1.8492e+09
[7]	valid_0's l2: 1.6532e+09
[8]	valid_0's l2: 1.49e+09
[9]	valid_0's l2: 1.34407e+09
[10]	valid_0's l2: 1.22399e+09
[11]	valid_0's l2: 1.13676e+09
[12]	valid_0's l2: 1.04572e+09
[13]	valid_0's l2: 9.47713e+08
[14]	valid_0's l2: 8.7168e+08
[15]	valid_0's l2: 8.16255e+08
[16]	valid_0's l2: 7.62894e+08
[17]	valid_0's l2: 7.14235e+08
[18]	valid_0's l2: 6.77013e+08
[19]	valid_0's l2: 6.53477e+08
[20]	valid_0's l2: 6.20648e+08
[21]	valid_0's l2: 5.94531e+08
[22]	valid_0's l2: 5.70285e+08
[23]	valid_0's l2: 5.50895e+08
[24]	valid_0's l2: 5.37362e+08
[25]	valid_0's l2:

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[51]	valid_0's l2: 4.02187e+08
[52]	valid_0's l2: 4.00771e+08
[53]	valid_0's l2: 4.00314e+08
[54]	valid_0's l2: 3.97452e+08
[55]	valid_0's l2: 3.97631e+08
[56]	valid_0's l2: 3.95873e+08
[57]	valid_0's l2: 3.94542e+08
[58]	valid_0's l2: 3.90672e+08
[59]	valid_0's l2: 3.90919e+08
[60]	valid_0's l2: 3.90251e+08
[61]	valid_0's l2: 3.86436e+08
[62]	valid_0's l2: 3.86624e+08
[63]	valid_0's l2: 3.83675e+08
[64]	valid_0's l2: 3.83414e+08
[65]	valid_0's l2: 3.81863e+08
[66]	valid_0's l2: 3.81902e+08
[67]	valid_0's l2: 3.80679e+08
[68]	valid_0's l2: 3.79744e+08
[69]	valid_0's l2: 3.80455e+08
[70]	valid_0's l2: 3.8128e+08
[71]	valid_0's l2: 3.79586e+08
[72]	valid_0's l2: 3.79973e+08
[73]	valid_0's l2: 3.79807e+08
[74]	valid_0's l2: 3.79169e+08
[75]	valid_0's l2: 3.76813e+08
[76]	valid_0's l2: 3.76188e+08
[77]	valid_0's l2: 3.76454e+08
[78]	valid_0's l2: 3.75737e+08
[79]	valid_0's l2: 3.74741e+08
[80]	valid_0's l2: 3.74434e+08
[81]	valid_0's l2: 3.72834e+08
[82]	valid_0's l2: 3.72529e+08
[83]	vali

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


: model1 rmse = 27099.427582134824
: model2 rmse = 26034.02178712582
: model3 rmse = 26834.036356213976
: model4 rmse = 26467.0061165277
: average all models rmse = 26533.48516192945
fold: 3
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=1e-10, subsample=1.0 will be ignored. Current value: bagging_fraction=1e-10
[1]	valid_0's l2: 5.26212e+09
[2]	valid_0's l2: 4.5977e+09
[3]	valid_0's l2: 3.99951e+09
[4]	valid_0's l2: 3.59134e+09
[5]	valid_0's l2: 3.09926e+09
[6]	valid_0's l2: 2.72381e+09
[7]	valid_0's l2: 2.41218e+09
[8]	valid_0's l2: 2.16164e+09
[9]	valid_0's l2: 1.95361e+09
[10]	valid_0's l2: 1.81945e+09
[11]	valid_0's l2: 1.63212e+09
[12]	valid_0's l2: 1.51125e+09
[13]	valid_0's l2: 1.38727e+09
[14]	valid_0's l2: 1.29085e+09
[15]	valid_0's l2: 1.20048e+09
[16]	valid_0's l2: 1.13856e+09
[17]	valid_0's l2: 1.08584e+09
[18]	valid_0's l2: 1.033e+09
[19]	valid_0's l2:

...

: model1 rmse = 25707.91748818708
: model2 rmse = 24669.2390751336
: model3 rmse = 25016.358054521104
: model4 rmse = 25190.218610601427
: average all models rmse = 25189.125057974034
fold: 4
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=1e-10, subsample=1.0 will be ignored. Current value: bagging_fraction=1e-10
[1]	valid_0's l2: 4.69027e+09
[2]	valid_0's l2: 3.95154e+09
[3]	valid_0's l2: 3.34333e+09
[4]	valid_0's l2: 2.88453e+09
[5]	valid_0's l2: 2.53079e+09
[6]	valid_0's l2: 2.17258e+09
[7]	valid_0's l2: 1.86966e+09
[8]	valid_0's l2: 1.62973e+09
[9]	valid_0's l2: 1.45416e+09
[10]	valid_0's l2: 1.2928e+09
[11]	valid_0's l2: 1.15148e+09
[12]	valid_0's l2: 1.02246e+09
[13]	valid_0's l2: 9.11628e+08
[14]	valid_0's l2: 8.38149e+08
[15]	valid_0's l2: 7.82642e+08
[16]	valid_0's l2: 7.18213e+08
[17]	valid_0's l2: 6.58274e+08
[18]	valid_0's l2: 6.12255e+08
[19]	valid_0's 

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


: model1 rmse = 18354.439716295914
: model2 rmse = 17159.091886827373
: model3 rmse = 18572.060591305577
: model4 rmse = 17787.5975061486
: average all models rmse = 17767.043036423962
fold: 5
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=1e-10, subsample=1.0 will be ignored. Current value: bagging_fraction=1e-10
[1]	valid_0's l2: 3.68033e+09
[2]	valid_0's l2: 3.11953e+09
[3]	valid_0's l2: 2.70667e+09
[4]	valid_0's l2: 2.35635e+09
[5]	valid_0's l2: 2.07378e+09
[6]	valid_0's l2: 1.81363e+09
[7]	valid_0's l2: 1.62568e+09
[8]	valid_0's l2: 1.43794e+09
[9]	valid_0's l2: 1.30509e+09
[10]	valid_0's l2: 1.1836e+09
[11]	valid_0's l2: 1.08837e+09
[12]	valid_0's l2: 1.01058e+09
[13]	valid_0's l2: 9.22788e+08
[14]	valid_0's l2: 8.53899e+08
[15]	valid_0's l2: 7.95087e+08
[16]	valid_0's l2: 7.53211e+08
[17]	valid_0's l2: 7.0407e+08
[18]	valid_0's l2: 6.74666e+08
[19]	valid_0's 

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[88]	valid_0's l2: 3.9765e+08
[89]	valid_0's l2: 3.98402e+08
[90]	valid_0's l2: 3.98476e+08
[91]	valid_0's l2: 3.98632e+08
[92]	valid_0's l2: 3.97718e+08
[93]	valid_0's l2: 3.98237e+08
[94]	valid_0's l2: 3.97857e+08
[95]	valid_0's l2: 3.97568e+08
[96]	valid_0's l2: 3.98026e+08
[97]	valid_0's l2: 3.97308e+08
[98]	valid_0's l2: 3.97482e+08
[99]	valid_0's l2: 3.97306e+08
[100]	valid_0's l2: 3.97154e+08
: model1 rmse = 19928.7278007983
: model2 rmse = 19848.95512302956
: model3 rmse = 18924.712038978374
: model4 rmse = 20271.27828430776
: average all models rmse = 20016.320402711874
fold: 1, rmse: 17951.29445332338
fold: 2, rmse: 26198.963045535504
fold: 3, rmse: 24936.996627786688
fold: 4, rmse: 17354.15767054611
fold: 5, rmse: 19293.23550108242


In [7]:
#scores = -1 * cross_val_score(my_model_2, X, y,
#                              cv=5,
#                              scoring='neg_mean_absolute_error')
#print("Average MAE score (across experiments):")
#print(scores.mean())

In [8]:
params = {
          'alpha_1' : [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
          'lambda_1' : [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
           }

xgb1 = BayesianRidge()



gcv = GridSearchCV(xgb1, param_grid=params, cv=4, n_jobs=-1)
gcv.fit(X, y)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",gcv.best_estimator_)
print("\n The best score across ALL searched params:\n",gcv.best_score_)
print("\n The best parameters across ALL searched params:\n",gcv.best_params_)


 Results from Grid Search 

 The best estimator across ALL searched params:
 BayesianRidge(alpha_1=0.5, lambda_1=0.01)

 The best score across ALL searched params:
 0.9056286803183881

 The best parameters across ALL searched params:
 {'alpha_1': 0.5, 'lambda_1': 0.01}


In [9]:
print(len(stacked_predictions))

1444


In [10]:


# Submit
#predsSub = my_model_2.predict(X_sub)
output = pd.DataFrame({'Id': X_sub.Id,
                       'SalePrice': final_prediction})
output.to_csv('submission.csv', index=False)

